In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from langchain_google_genai import embeddings
import uuid
import os
from dotenv import load_dotenv
from qdrant_client.models import (
    VectorParams, SparseVectorParams, Distance
)
from sentence_transformers import SentenceTransformer
import re
from qdrant_client.models import PointStruct

C:\Users\GIGABYTE\PycharmProjects\VectorsHackthon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_csv("walmart-products.csv")

In [3]:
dataset.head(10)

,timestamp,url,final_price,sku,currency,gtin,specifications,image_urls,top_reviews,rating_stars,...,sizes,colors,seller,other_attributes,customer_reviews,ingredients,initial_price,discount,ingredients_full,categories
0,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Laura-Mercier-Cavia...,22.90,173530386,USD,7.361501e+11,"[{""name"":""Brand"",""value"":""Laura Mercier""},{""na...","[""https://i5.walmartimages.com/seo/Laura-Merci...","{""negative"":{},""positive"":{}}","{""five_stars"":2,""four_stars"":3,""two_stars"":1}",...,[],"[""Sugar Frost"",""Tuxedo""]",Wal███t.c███,"[{""name"":""Instructions"",""value"":""Apply directl...","[{""name"":""Jac███"",""rating"":5,""review"":""My only...","Cyclopentasiloxane, trimethylsiloxysilicate, s...",NaN,NaN,"[{""type"":""Ingredients"",""values"":""Cyclopentasil...","[""Beauty"",""Makeup"",""Eye Makeup"",""Eye Shadow"",""..."
1,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Exultantex-Grey-Bla...,47.88,430528189,USD,7.710779e+11,"[{""name"":""Brand"",""value"":""Exultantex""},{""name""...","[""https://i5.walmartimages.com/seo/Exultantex-...","{""negative"":{""rating"":1,""review"":""Color not ac...","{""five_stars"":47,""four_stars"":4,""one_star"":4,""...",...,"[""50\"" x 54\"""",""50\"" x 63\"""",""50\"" x 84\"""",""50...","[""Black"",""Blue"",""Green"",""Gray"",""Natural(Ivory)...",Exu███nte███ome███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Dana"",""rating"":5,""review"":""I love th...",NaN,70.80,$22.92,NaN,"[""Home"",""Decor"",""Curtains & Window Treatments""..."
2,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Jessica-London-Wome...,33.24,6013308220,USD,4.651920e+11,"[{""name"":""Features"",""value"":""Easy Care""},{""nam...","[""https://i5.walmartimages.com/seo/Jessica-Lon...","{""negative"":{},""positive"":{}}","{""five_stars"":2}",...,"[""S"",""M"",""L"",""1X"",""2X"",""3X""]","[""Aqua Sea"",""Dark Olive Green"",""Dark Sapphire""...",Ful███aut███ran███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Bev███y F███"",""rating"":5,""review"":""V...",NaN,37.99,$4.75,NaN,"[""Clothing"",""Womens Plus"",""Plus Size Tops"",""Pl..."
3,2024-08-24 00:00:00.000,https://www.walmart.com/ip/100-Cotton-King-Per...,49.99,161657830,USD,8.407081e+11,"[{""name"":""Brand"",""value"":""Simply Put""},{""name""...","[""https://i5.walmartimages.com/asr/d88fe658-b9...","{""negative"":{""rating"":1,""review"":""[This review...","{""five_stars"":78,""four_stars"":17,""one_star"":2,...",...,"[""Queen"",""King""]","[""Beige"",""Blue"",""Gray"",""Spa Blue""]",IC ███bal███c,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Jes███oge███"",""rating"":5,""review"":""[...",NaN,71.99,$22.00,NaN,"[""Home"",""Bedding"",""Duvet Covers"",""King Duvet C..."
4,2024-08-25 00:00:00.000,https://www.walmart.com/ip/Disney-Boys-Graphic...,12.99,5397071399,USD,4.600086e+11,"[{""name"":""Country of Origin - Textiles"",""value...","[""https://i5.walmartimages.com/seo/Disney-Boys...","{""negative"":{""rating"":1,""review"":""This size sh...","{""five_stars"":12,""one_star"":1,""three_stars"":1,...",...,"[""2T"",""3T"",""3T-4T"",""3-4 Years"",""4-5 Years"",""4-...","[""1#Yellow Micky"",""2#Pink Minnie"",""3#Blue Dona...",Pat███,NaN,"[{""name"":""Mary"",""rating"":5,""review"":""Love thes...",NaN,29.00,$16.01,NaN,"[""Clothing"",""Kids Clothing"",""Boys Clothing"",""B..."
5,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Women-s-Vanity-Fair...,28.00,512809025,USD,8.362613e+10,"[{""name"":""Clothing Size"",""value"":""40C""},{""name...","[""https://i5.walmartimages.com/seo/Women-s-Van...","{""negative"":{""rating"":2,""review"":""[This review...","{""five_stars"":63,""four_stars"":21,""one_star"":5,...",...,"[""34B"",""34C"",""34D"",""34DD"",""36 D"",""36C"",""36DD"",...","[""Cappuccino"",""Damask Neutral"",""Gray"",""In The ...",Her███m a███His██████,"[{""name"":""Fabric Care Instructions"",""value"":""H...","[{""name"":""Tfr███7"",""rating"":

In [4]:
dataset.isna().sum()

timestamp                   0
url                         0
final_price                 0
sku                         0
currency                    0
gtin                        1
specifications              0
image_urls                  0
top_reviews                 0
rating_stars                0
related_pages               9
available_for_delivery      0
available_for_pickup        0
brand                       0
breadcrumbs                 0
category_ids                0
review_count                0
description                 1
product_id                  0
product_name                0
review_tags                 0
category_url                0
category_name               0
category_path               0
root_category_url           0
root_category_name          0
upc                         1
tags                      213
main_image                  0
rating                      0
unit_price                899
unit                      899
aisle                     950
free_retur

In [5]:
print(dataset["unit_price"][0])
print(dataset["initial_price"][0])
print(dataset["final_price"][0])
print(dataset["discount"][0])

22.9
nan
22.9
nan


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   timestamp               1000 non-null   object 
 1   url                     1000 non-null   object 
 2   final_price             1000 non-null   float64
 3   sku                     1000 non-null   int64  
 4   currency                1000 non-null   object 
 5   gtin                    999 non-null    float64
 6   specifications          1000 non-null   object 
 7   image_urls              1000 non-null   object 
 8   top_reviews             1000 non-null   object 
 9   rating_stars            1000 non-null   object 
 10  related_pages           991 non-null    object 
 11  available_for_delivery  1000 non-null   bool   
 12  available_for_pickup    1000 non-null   bool   
 13  brand                   1000 non-null   object 
 14  breadcrumbs             1000 non-null   o

In [7]:
len(dataset)

1000

In [8]:
df = dataset.copy()

In [9]:
df["initial_price"] = df["initial_price"].fillna(df["final_price"])

In [10]:
df.isna().sum()

timestamp                   0
url                         0
final_price                 0
sku                         0
currency                    0
gtin                        1
specifications              0
image_urls                  0
top_reviews                 0
rating_stars                0
related_pages               9
available_for_delivery      0
available_for_pickup        0
brand                       0
breadcrumbs                 0
category_ids                0
review_count                0
description                 1
product_id                  0
product_name                0
review_tags                 0
category_url                0
category_name               0
category_path               0
root_category_url           0
root_category_name          0
upc                         1
tags                      213
main_image                  0
rating                      0
unit_price                899
unit                      899
aisle                     950
free_retur

In [11]:
df = df.dropna(subset=["description"])

In [12]:
df["details"] = (
    "product : " + df['product_name'].astype(str) +
    " category : " + df['category_name'].astype(str) +
    " details : " + df['description'].astype(str)
)

In [13]:
from qdrant_client import models
from fastembed import TextEmbedding

dense_model = SentenceTransformer("all-MiniLM-L6-v2")

In [14]:
client = QdrantClient(url="http://localhost:6333")
from qdrant_client.models import (
    VectorParams, SparseVectorParams, Distance, PointStruct
)

client.recreate_collection(
    collection_name="products",
    vectors_config={
        "text-dense": VectorParams(
            size=384,
            distance=Distance.COSINE
        )
    },
    sparse_vectors_config={
        "text-sparse": SparseVectorParams(
            modifier=models.Modifier.IDF,
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    }
)

dense_vectors = dense_model.encode(
    df["details"].tolist(),
    show_progress_bar=True,
    normalize_embeddings=True
)
def clean_price(price_str):
    if price_str is None:
        return None
    cleaned = re.sub(r"[^\d.,]", "", str(price_str))
    cleaned = cleaned.replace(",", ".")
    try:
        return float(cleaned)
    except:
        return None

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_26164\3675459053.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
Batches: 100%|██████████| 32/32 [00:18<00:00,  1.69it/s]


In [15]:
payloads = [
    {
        "actual_price": clean_price(row.initial_price),
        "discounted_price": clean_price(row.final_price),
        "category": row.category_name,
        "rating": row.rating,
        "image_url": row.main_image,
        "product_url" : row.url,
    }
    for row in df.itertuples()
]

In [16]:
points = []
texts = df["details"].tolist()
avg_doc_length = sum(len(text.split()) for text in texts) / len(texts)
for i, row in enumerate(df.itertuples()):
    points.append(
        PointStruct(
            id=i,
            vector={
                "text-dense": dense_vectors[i],
                "text-sparse": models.Document(
                    text=texts[i],
                    model="prithivida/Splade_PP_en_v1",
                    options={"avg_len": avg_doc_length}
                )
            },
            payload=payloads[i]
        )
    )

client.upsert(
    collection_name="products",
    points=points
)

Fetching 5 files:  20%|██        | 1/5 [00:01<00:04,  1.18s/it]C:\Users\GIGABYTE\PycharmProjects\VectorsHackthon\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GIGABYTE\AppData\Local\Temp\fastembed_cache\models--Qdrant--Splade_PP_en_v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 f

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
# Get collection info
collection_info = client.get_collection("products")
print(collection_info)

# Look for vector config
print("\nVector configs:")
print(collection_info.config.params.vectors)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=999 points_count=999 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors={'text-dense': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'text-sparse': SparseVectorParams(index=SparseIndexParams(full_scan_threshold=None, on_disk=False, datatype=None), modifier=<Modifier.IDF: 'idf'>)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_thresh

In [24]:
from fastembed import SparseTextEmbedding, TextEmbedding
sparse_model = SparseTextEmbedding(model_name="prithivida/Splade_PP_en_v1")
query_text = "noise cancelling headphones"
query_dense_vector = dense_model.encode(query_text).tolist()
query_sparse_vector = list(sparse_model.embed([query_text]))[0]
results = client.query_points(
        collection_name="products",
        prefetch=[
            # Dense Search
            models.Prefetch(
                query=query_dense_vector,
                using="text-dense",
                limit=20,
            ),
            # Sparse Search
            models.Prefetch(
                query=models.SparseVector(
                    indices=query_sparse_vector.indices,
                    values=query_sparse_vector.values
                ),
                using="text-sparse",
                limit=20,
            ),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=10
    )

In [25]:
for i, point in enumerate(results.points, 1):
    print(f"\n--- Result {i} ---")
    print(f"Score: {point.score}")
    print(f"ID: {point.id}")
    print(f"Payload: {point.payload}")


--- Result 1 ---
Score: 0.6666667
ID: 640
Payload: {'actual_price': 28.89, 'discounted_price': 18.95, 'category': 'Room Darkening Curtains', 'rating': 3.9, 'image_url': '"https://i5.walmartimages.com/seo/Eclipse-Dayton-Solid-Blackout-Rod-Pocket-Energy-Efficient-Curtain-Panel-Chocolate-42-x63_4f6e4fc0-d427-4ee8-bedc-5dec918672b9.0fe5258f4306def3d455f68387a77236.jpeg"', 'product_url': 'https://www.walmart.com/ip/Eclipse-Dayton-Solid-Blackout-Rod-Pocket-Energy-Efficient-Curtain-Panel-Chocolate-42-x63/22406141'}

--- Result 2 ---
Score: 0.5
ID: 403
Payload: {'actual_price': 17.99, 'discounted_price': 12.99, 'category': 'Wireless Bras', 'rating': 4.4, 'image_url': '"https://i5.walmartimages.com/seo/Playtex-18-Hour-Ultimate-Lift-Support-Wireless-Bra-Nude-48D-Women-s_8ea02c2a-4f1c-4b92-a446-3d514e960394.03deaede9ab5ffa5c10bdcf0d57b575f.jpeg"', 'product_url': 'https://www.walmart.com/ip/Playtex-18-Hour-Ultimate-Lift-Support-Wireless-Bra-Nude-48D-Women-s/494669579'}

--- Result 3 ---
Score: 0.